In [6]:
import cv2
import mediapipe as mp
import numpy as np

# Initialisation MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Calcul de l'angle entre 3 points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
              np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return angle

# Capture webcam
cap = cv2.VideoCapture(0)

counter = 0
stage = None  # up / down

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Conversion couleur
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Points du bras (droit)
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]

        # Angle du coude
        angle = calculate_angle(shoulder, elbow, wrist)

        # Logique de comptage
        if angle > 160:
            stage = "up"
        if angle < 90 and stage == "up":
            stage = "down"
            counter += 1

        # Affichage angle
        cv2.putText(image, str(int(angle)),
                    tuple(np.multiply(elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

    # Affichage compteur
    cv2.rectangle(image, (0,0), (225,73), (0,0,0), -1)
    cv2.putText(image, 'Pompes', (10,20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
    cv2.putText(image, str(counter), (10,60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)

    # Dessin squelette
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pompe Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
